In [1]:
#import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import requests
import json

In [2]:
#load in csv
olympics_data = pd.read_csv("Summer-Olympic-medals-1976-to-2008.csv", encoding="latin-1")
olympics_data.head()

,City,Year,Sport,Discipline,Event,Athlete,Gender,Country_Code,Country,Event_gender,Medal
0,Montreal,1976.0,Aquatics,Diving,3m springboard,"KÖHLER, Christa",Women,GDR,East Germany,W,Silver
1,Montreal,1976.0,Aquatics,Diving,3m springboard,"KOSENKOV, Aleksandr",Men,URS,Soviet Union,M,Bronze
2,Montreal,1976.0,Aquatics,Diving,3m springboard,"BOGGS, Philip George",Men,USA,United States,M,Gold
3,Montreal,1976.0,Aquatics,Diving,3m springboard,"CAGNOTTO, Giorgio Franco",Men,ITA,Italy,M,Silver
4,Montreal,1976.0,Aquatics,Diving,10m platform,"WILSON, Deborah Keplar",Women,USA,United States,W,Bronze


In [36]:
olympics_data["Medal Point"]

0         
1         
2         
3         
4         
        ..
15428     
15429     
15430     
15431     
15432     
Name: Medal Point, Length: 15433, dtype: object

In [37]:
olympics_data

,City,Year,Sport,Discipline,Event,Athlete,Gender,Country_Code,Country,Event_gender,Medal,Medal Point
0,Montreal,1976.0,Aquatics,Diving,3m springboard,"KÖHLER, Christa",Women,GDR,East Germany,W,Silver,
1,Montreal,1976.0,Aquatics,Diving,3m springboard,"KOSENKOV, Aleksandr",Men,URS,Soviet Union,M,Bronze,
2,Montreal,1976.0,Aquatics,Diving,3m springboard,"BOGGS, Philip George",Men,USA,United States,M,Gold,
3,Montreal,1976.0,Aquatics,Diving,3m springboard,"CAGNOTTO, Giorgio Franco",Men,ITA,Italy,M,Silver,
4,Montreal,1976.0,Aquatics,Diving,10m platform,"WILSON, Deborah Keplar",Women,USA,United States,W,Bronze,
...,...,...,...,...,...,...,...,...,...,...,...,...
15428,Beijing,2008.0,Wrestling,Wrestling Gre-R,66 - 74kg,"GUENOT, Christophe",Men,FRA,France,M,Bronze,
15429,Beijing,2008.0,Wrestling,Wrestling Gre-R,66 - 74kg,"KVIRKELIA, Manuchar",Men,GEO,Georgia,M,Gold,
15430,Beijing,2008.0,Wrestling,Wrestling Gre-R,55 - 60kg,"RAHIMOV, Vitaliy",Men,AZE,Azerbaijan,M,Silver,
15431,Beijing,2008.0,Wrestling,Wrestling Gre-R,60 - 66kg,"GUENOT, Steeve",Men,FRA,France,M,Gold,


In [112]:
#Loop through to calculate points for medals


medals = olympics_data["Medal"].value_counts()
medal_count= []


for medal in medals:
    
    count = 1
    
    if medal == 'Gold':
        medal_count= (count + 2)
        
    elif  medal == 'Silver':
        medal_count= (count + 1)
        
    else:
        medal_count= count

In [113]:
olympics_data["Medal Point"]= medal_count

In [114]:
olympics_data

,City,Year,Sport,Discipline,Event,Athlete,Gender,Country_Code,Country,Event_gender,Medal,Medal Point
0,Montreal,1976.0,Aquatics,Diving,3m springboard,"KÖHLER, Christa",Women,GDR,East Germany,W,Silver,1
1,Montreal,1976.0,Aquatics,Diving,3m springboard,"KOSENKOV, Aleksandr",Men,URS,Soviet Union,M,Bronze,1
2,Montreal,1976.0,Aquatics,Diving,3m springboard,"BOGGS, Philip George",Men,USA,United States,M,Gold,1
3,Montreal,1976.0,Aquatics,Diving,3m springboard,"CAGNOTTO, Giorgio Franco",Men,ITA,Italy,M,Silver,1
4,Montreal,1976.0,Aquatics,Diving,10m platform,"WILSON, Deborah Keplar",Women,USA,United States,W,Bronze,1
...,...,...,...,...,...,...,...,...,...,...,...,...
15428,Beijing,2008.0,Wrestling,Wrestling Gre-R,66 - 74kg,"GUENOT, Christophe",Men,FRA,France,M,Bronze,1
15429,Beijing,2008.0,Wrestling,Wrestling Gre-R,66 - 74kg,"KVIRKELIA, Manuchar",Men,GEO,Georgia,M,Gold,1
15430,Beijing,2008.0,Wrestling,Wrestling Gre-R,55 - 60kg,"RAHIMOV, Vitaliy",Men,AZE,Azerbaijan,M,Silver,1
15431,Beijing,2008.0,Wrestling,Wrestling Gre-R,60 - 66kg,"GUENOT, Steeve",Men,FRA,France,M,Gold,1


In [90]:
#Groupby Country & Sport

grouped_multiple = olympics_data.groupby(['Country_Code', 'Sport'])

print(grouped_multiple)

In [91]:
grouped_multiple.describe()

Year                                                 \
                       count         mean       std     min     25%     50%   
Country_Code Sport                                                            
AFG          Taekwondo   1.0  2008.000000       NaN  2008.0  2008.0  2008.0   
AHO          Sailing     1.0  1988.000000       NaN  1988.0  1988.0  1988.0   
ALG          Athletics   6.0  1998.000000  3.346640  1992.0  1997.0  2000.0   
             Boxing      6.0  1992.000000  6.693280  1984.0  1986.0  1994.0   
             Judo        2.0  2008.000000  0.000000  2008.0  2008.0  2008.0   
...                      ...          ...       ...     ...     ...     ...   
YUG          Wrestling   9.0  1982.222222  4.055175  1976.0  1980.0  1984.0   
ZAM          Athletics   1.0  1996.000000       NaN  1996.0  1996.0  1996.0   
             Boxing      1.0  1984.000000       NaN  1984.0  1984.0  1984.0   
ZIM          Aquatics    7.0  2006.285714  2.138090  2004.0  2004.0  2008.0   
             Hockey     16.0  1980.000000  0.000000  1980.0  1980.0  1980.0   

                                       Medal Point                           \
                           75%     max       count mean  std  min  25%  50%   
Country_Code Sport                                                            
AFG          Taekwondo  2008.0  2008.0         1.0  2.0  NaN  2.0  2.0  2.0   
AHO          Sailing    1988.0  1988.0         1.0  2.0  NaN  2.0  2.0  2.0   
ALG          Athletics  2000.0  2000.0         6.0  2.0  0.0  2.0  2.0  2.0   
             Boxing     1996.0  2000.0         6.0  2.0  0.0  2.0  2.0  2.0   
             Judo       2008.0  2008.0         2.0  2.0  0.0  2.0  2.0  2.0   
...                        ...     ...         ...  ...  ...  ...  ...  ...   
YUG          Wrestling  1984.0  1988.0         9.0  2.0  0.0  2.0  2.0  2.0   
ZAM          Athletics  1996.0  1996.0         1.0  2.0  NaN  2.0  2.0  2.0   
             Boxing     1984.0  1984.0         1.0  2.0  NaN  2.0  2.0  2.0   
ZIM          Aquatics   2008.0  2008.0         7.0  2.0  0.0  2.0  2.0  2.0   
             Hockey     1980.0  1980.0        16.0  2.0  0.0  2.0  2.0  2.0   

                                  
                        75%  max  
Country_Code Sport                
AFG          Taekwondo  2.0  2.0  
AHO          Sailing    2.0  2.0  
ALG          Athletics  2.0  2.0  
             Boxing     2.0  2.0  
             Judo       2.0  2.0  
...                     ...  ...  
YUG          Wrestling  2.0  2.0  
ZAM          Athletics  2.0  2.0  
             Boxing     2.0  2.0  
ZIM          Aquatics   2.0  2.0  
             Hockey     2.0  2.0  

[829 rows x 16 columns]

In [127]:
#Create dataframe

medals_by_sport = medals_x_country_yr['Medal'].count()

medals_by_sport

point_total= medals_x_country_yr['Medal Point'].sum()

In [128]:
summary_stats = pd.DataFrame({'By Sport': medals_by_sport, 'Point Total': point_total})

summary_stats

By Sport  Point Total
Country_Code Year                         
AFG          2008.0         1            1
AHO          1988.0         1            1
ALG          1984.0         2            2
             1992.0         2            2
             1996.0         3            3
...                       ...          ...
ZAM          1984.0         1            1
             1996.0         1            1
ZIM          1980.0        16           16
             2004.0         3            3
             2008.0         4            4

[559 rows x 2 columns]

In [119]:
#Groupby Country & Year

medals_x_country_yr = olympics_data.groupby(['Country_Code', 'Year'])

print(medals_x_country_yr)

In [120]:
medals_x_country_yr.describe()

Medal Point                                   
                          count mean  std  min  25%  50%  75%  max
Country_Code Year                                                 
AFG          2008.0         1.0  1.0  NaN  1.0  1.0  1.0  1.0  1.0
AHO          1988.0         1.0  1.0  NaN  1.0  1.0  1.0  1.0  1.0
ALG          1984.0         2.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
             1992.0         2.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
             1996.0         3.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
...                         ...  ...  ...  ...  ...  ...  ...  ...
ZAM          1984.0         1.0  1.0  NaN  1.0  1.0  1.0  1.0  1.0
             1996.0         1.0  1.0  NaN  1.0  1.0  1.0  1.0  1.0
ZIM          1980.0        16.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
             2004.0         3.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
             2008.0         4.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0

[559 rows x 8 columns]

In [121]:
#Create dataframe

medals_by_yr = medals_x_country_yr['Medal'].count()

medals_by_yr

Country_Code  Year  
AFG           2008.0     1
AHO           1988.0     1
ALG           1984.0     2
              1992.0     2
              1996.0     3
                        ..
ZAM           1984.0     1
              1996.0     1
ZIM           1980.0    16
              2004.0     3
              2008.0     4
Name: Medal, Length: 559, dtype: int64

In [122]:
year_stats = pd.DataFrame({'By Sport': medals_by_yr, 'Point Total': point_total})

year_stats

By Sport  Point Total
Country_Code Year                         
AFG          2008.0         1            1
AHO          1988.0         1            1
ALG          1984.0         2            2
             1992.0         2            2
             1996.0         3            3
...                       ...          ...
ZAM          1984.0         1            1
             1996.0         1            1
ZIM          1980.0        16           16
             2004.0         3            3
             2008.0         4            4

[559 rows x 2 columns]